# Michael H. Terrefortes Rosado
# CCOM6995
## Data visualization final project
### Description: Allow user to binarize a gene expression dataset and explore the process. 

# Code for BASC A algorithm

In [4]:
from dash import Dash, dash_table, dcc, html, Input, Output, callback
import pandas as pd
import numpy as np
import plotly.graph_objects as go
import math
import matplotlib as plt

def Y_a_b(genes, a, b):
  return np.mean(genes[a:b])

def C_a_b(genes, a, b):
  mean = Y_a_b(genes, a, b+1)
  return sum( (np.array(genes[a:b+1]) - mean) ** 2 )

def determine_h(P, i, j, genes):
  N = len(genes)

  if (i == 0 and j > 0):
    return Y_a_b(genes, P[i][j], P[i+1][j]) - Y_a_b(genes, 0, P[i][j]);
  elif (i == j and j > 0):
    return Y_a_b(genes, P[i][j], N) - Y_a_b(genes, P[i-1][j], P[i][j]);
  elif (i == 0 and j == 0):
    return Y_a_b(genes, P[i][j], N) - Y_a_b(genes, 0, P[i][j]);
  else:
    return Y_a_b(genes, P[i][j], P[i+1][j]) - Y_a_b(genes, P[i-1][j], P[i][j]);

def BASC_A(gene):
    gene_og = gene
    gene = np.sort(gene)
    N = len(gene)

    cost_matrix = [[0 for _ in range(N - 1)] for _ in range(N)]
    ind_matrix = [[0 for _ in range(N - 2)] for _ in range(N - 1)]
    P = [[0 for _ in range(N - 2)] for _ in range(N - 2)]

    # Step 1: Compute a Series of Step Function

    # initialization C_i_(0) = c_i_N
    # calculate first cost matrix column with no intermidiate break points
    for i in range(N):
      cost_matrix[i][0] = C_a_b(gene, i, N)

    # Algorithm 1: Calculate optimal step functions
    for j in range(N-2):
      for i in range(N-j-1):
        min_value = math.inf
        min_index = math.inf

        for d in range(N-j-1):
          curr_value = C_a_b(gene, i, d) + cost_matrix[d+1][j]

          if(curr_value < min_value):
            min_value = curr_value
            min_index = d

        cost_matrix[i][j+1] = min_value
        ind_matrix[i][j] = min_index + 1

    #  Algorithm 2: Compute the break points of all optimal step functions
    for j in range(N-2):
      z = j
      P[0][j] = ind_matrix[0][z]
      if(j > 0):
        z = z - 1
        for i in range(1, j+1):
          P[i][j] = ind_matrix[P[i-1][j]][z]
          z = z - 1

    # Step 2: Find Strongest Discontinuity in Each Step Function
    v = [0] * (N-2)

    for j in range(N-2):
      max_value = -math.inf
      max_index = j
      for i in range(j+1):
        h = determine_h(P, i, j, gene)
        z = (gene[P[i][j]] + gene[P[i][j]-1]) / 2
        e = sum( (np.array(gene) - z) ** 2 )
        q_score = h / e
        if(q_score > max_value):
          max_value = q_score
          max_index = i

      v[j] = P[max_index][j]

    # Step 3: Estimate Location and Variation of the Strongest Discontinuities
    thr = (gene[round(np.median(v))-1] + gene[round(np.median(v))]) / 2

    return thr, P


# Code for K-means

In [5]:
from sklearn.cluster import KMeans

def K_Means(genes):
    data = np.array(genes).reshape(-1, 1)
    kmeans = KMeans(n_clusters=2)
    kmeans.fit(data)
    c=kmeans.labels_
    genes = np.array(genes)
    groupOne = genes[c==1]
    groupZero = genes[c==0]
    
    thr1 = np.mean(groupOne)
    thr2 = np.mean(groupZero)
    
    thr = (thr1 + thr2) / 2

    return thr

# Code for Stepminer

In [6]:
def getSSTOT(x, n, xmean):
    m = 0
    for i in range(n):
        m = m + (x[i] - xmean)**2
    return m


def onestep(x):
    
    n = len(x)
    #step = 0
    xmean = np.mean(x)
    SSTOT = getSSTOT(x, n, xmean)
    
    print(xmean)
    print(SSTOT)
    
    SSEmin = SSTOT
    
    for i in range(n-1):
        leftMean = np.mean(x[0:i+1])
    
        rightMean = np.mean(x[i+1:n])
        
        SSE = 0
        
        for j in range(n):
            if j < i+1:
                SSE = SSE + (x[j] - leftMean)**2
            else:
                SSE = SSE + (x[j] - rightMean)**2
                    
        
        if SSEmin > SSE:
            SSEmin = SSE
            #print("1:",SSEmin1)
                
            t = (leftMean + rightMean)/2
        
    
    return t

# Code for Shmulevich

In [7]:
def shmulevich(x):
    
    n = len(x)
    s = np.sort(x)
    d = np.empty(n)
    
    for i in range(n-2):
        d[i] = s[i+1] - s[i]
    
    t = (s[n-1] - s[0])/(n-1)
    
    mn = s[n-1]
    index = 0
    
    for i in range(n-1):
        if d[i] > t and d[i] < mn:
            mn = d[i]
            index = i
            
    z = s[index + 1]
   
    
    return z

# Interpolation Code

In [8]:
from scipy.interpolate import splev, splrep

def interpolation(gene, method, tolerance, limit):
    
    converge = False
    samples = 10
    thr_array = []
    tMin = 0
    tMax = 0
    thr = 0
    
    while(~converge and samples != limit):
        n = len(gene)
        x = np.arange(1, n + 1, dtype=int)
        y = gene

        xx = np.arange(1, n + 1, (n) / (samples))

        p = splrep(x, y, s=0)
        yy = splev(xx, p)
        
        if(method == 'K-Means'):
            thr = K_Means(yy)
            
        elif(method == 'BASC A'):
            thr, _ = BASC_A(yy)
            
        elif(method == 'Onestep'):
            thr = onestep(yy)
            
        else:
            thr = shmulevich(yy)
        
        thr_array.append(thr)
        
        n_thr = len(thr_array)
        
        for i in range(n_thr):
            for j in range(n_thr):
                if(i != j):
                    difference = abs(thr_array[i] - thr_array[j])
                    
                    if(difference <= tolerance):
                        converge = True
                        break
                        
            if(converge or samples != limit):
                tMin = min(thr_array)
                tMax = max(thr_array)
                break
                
        if(~converge):
            samples += 10
            
            
    
    return tMin, tMax
        

# Interpolation Until Merge Code

In [15]:
from scipy.interpolate import splev, splrep

def interpolationConverge(gene, method, tolerance):
    
    converge = False
    samples = 10
    thr_array = []
    tMin = 0
    tMax = 0
    thr = 0
    conver = 0
    #change later
    limit = 10000
    
    while(~converge and samples != limit):
        n = len(gene)
        x = np.arange(1, n + 1, dtype=int)
        y = gene

        xx = np.arange(1, n + 1, (n) / (samples))

        p = splrep(x, y, s=0)
        yy = splev(xx, p)
        
        if(method == 'K-Means'):
            thr = K_Means(yy)
            
        elif(method == 'BASC A'):
            thr, _ = BASC_A(yy)
            
        elif(method == 'Onestep'):
            thr = onestep(yy)
            
        else:
            thr = shmulevich(yy)
        
        thr_array.append(thr)
        
        n_thr = len(thr_array)
        
        for i in range(n_thr):
            for j in range(n_thr):
                if(i != j):
                    difference = abs(thr_array[i] - thr_array[j])
                    
                    if(difference <= tolerance):
                        converge = True
                        conver = thr_array[j]
                        break
                
        if(~converge):
            samples += 10

        
            


    tMin = min(thr_array)
    tMax = max(thr_array)
    
    return tMin, tMax, conver, samples




# Code for Dashboard 

In [11]:
import dash_bootstrap_components as dbc 

df = pd.read_csv('HIVIn(Matlab).csv')

col_names = {'basc_thr':[], 'kmeans_thr':[], 'onestep_thr':[], 'shmulevich_thr':[]}
final_df = pd.DataFrame(col_names)

app = Dash(__name__)
server = app.server

app.layout = dbc.Container([
    dbc.Row([
       dbc.Col([
            html.H4('Binarize Genes Visualization'), 
            html.Div([
                dash_table.DataTable(
                    id='datatable-interactivity',
                    columns=[
                        {"name": i, "id": i} for i in df.columns
                    ],
                    data=df.to_dict('records'),
                    column_selectable="single",
                    row_selectable="multi",
                    selected_columns=[],
                    selected_rows=[],
                    page_action="native",
                    page_current= 0,
                    page_size= 10,
                ),
                html.H5('Select option in order to show all the thresholds by each algorithm of each gene:'),
                dcc.Dropdown(
                            [{'label': 'Binarize All Genes', 'value':'all'}],
                            placeholder="Select to binarize all genes and get thresholds",
                            id="dropdown-binarize-all",
                            searchable=False),
                html.Div(id='binarize-all'),
                html.Hr(),
                html.Div(id='dropdown-methods')
            ])
       ], width=8)
    ], justify="center"),
    dbc.Row([
        dbc.Col([html.Div(id='heatmap-binarize')], width=5),
        dbc.Col([
            html.Div(id='select-gene-binarize'),
            html.Div(id='graph-gene-binarize'),
            html.Div(id='select-basc-discontinuity'),
            html.Div(id='graph-gene-discontinuity'),
            html.Div(id='interpolation-dropdown'),
            html.Div(id='tolerance-dropdown'),
            html.Div(id='interpolation-graph')
        ], width=5)
    ])
])

@app.callback(
    Output('binarize-all', 'children'),
    Input('dropdown-binarize-all', 'value'))
def binarize_all(all_rows):
    if all_rows is None:
        return None
    genes = df.values
    rows = df.shape[0]
    
    col_names = {'basc_thr':[], 'kmeans_thr':[], 'onestep_thr':[], 'shmulevich_thr':[]}
    
    for i in range(rows):
            k_means = K_Means(genes[i])
            basc_a, _ = BASC_A(genes[i])
            one_step = onestep(genes[i])
            shmulevich_ = shmulevich(genes[i])
            
            new_row = {'basc_thr':basc_a, 'kmeans_thr':k_means, 'onestep_thr':one_step, 'shmulevich_thr': shmulevich_}
            final_df.loc[len(final_df)] = new_row
        
    return [dash_table.DataTable(final_df.to_dict('records'), [{"name": i, "id": i} for i in final_df.columns],
                               page_size= 10), html.Button("Download CSV", id="btn_csv"),
        dcc.Download(id="download-dataframe-csv")]

@callback(
    Output("download-dataframe-csv", "data"),
    Input("btn_csv", "n_clicks"),
    prevent_initial_call=True,
)
def func(n_clicks):
    return dcc.send_data_frame(final_df.to_csv, "thr.csv")
    
@app.callback(
    Output('dropdown-methods', 'children'),
    Input('datatable-interactivity', 'selected_rows'))
def display_selected_data(selected_rows):
    if not selected_rows:
        return None
    return  [html.H5('Select binarization method to binarize selected rows and visualize:'), 
            dcc.Dropdown(
                ['All','BASC A', 'K-Means', 'Onestep', 'Shmulevich'],
                placeholder="Select binarization method",
                id="dropdown-method",
                searchable=False)]

@app.callback(
    Output('heatmap-binarize', 'children'),
    Input('dropdown-method', 'value'),
    Input('datatable-interactivity', 'selected_rows'), prevent_initial_call=True)
def heatmap_binarize(selected_method, selected_rows):
    if selected_rows is None:
        return None
    if selected_method is None:
        return None
    
    #selected = df.iloc[selected_rows]
    #gene = selected.values
    #sizeGene = len(gene)
    
    if(selected_method == "K-Means"):
        binarize_vect = []
        labels = []

        for row in selected_rows:
            selected = df.iloc[row]
            gene = selected.values
            sizeGene = len(gene)
            binarize = []
            labels.append("Gene " + str(row+1))
            thr = K_Means(gene)

            for j in range(sizeGene):
                if(gene[j] <= thr):
                    binarize.append(0) 
                else:
                    binarize.append(1)     

            binarize_vect.append(binarize) 
            
        selected = df.iloc[selected_rows]
        genes = selected.values
        
        data = go.Figure(data=go.Heatmap(
                    z=genes,
                    y = labels,
                    text=binarize_vect,
                    texttemplate="%{text}",
                    textfont={"size":20}))
        data.update_layout(
            title={
        'text': "Heatmap of Binarized Genes using K-Means",
            }
        ) 
        return dcc.Graph(figure=data)
        
    elif(selected_method == "BASC A"):   
        binarize_vect = []
        labels = []

        for row in selected_rows:
            selected = df.iloc[row]
            gene = selected.values
            sizeGene = len(gene)
            binarize = []
            labels.append("Gene " + str(row+1))
            thr, _ = BASC_A(gene)

            for j in range(sizeGene):
                if(gene[j] <= thr):
                    binarize.append(0) 
                else:
                    binarize.append(1)     

            binarize_vect.append(binarize) 
            
        selected = df.iloc[selected_rows]
        genes = selected.values
        
        data = go.Figure(data=go.Heatmap(
                    z=genes,
                    y = labels,
                    text=binarize_vect,
                    texttemplate="%{text}",
                    textfont={"size":20}))
        data.update_layout(
            title={
        'text': "Heatmap of Binarized Genes using BASC A",
            }
        )
        return dcc.Graph(figure=data)
    
    elif(selected_method == "Onestep"):
        binarize_vect = []
        labels = []

        for row in selected_rows:
            selected = df.iloc[row]
            gene = selected.values
            sizeGene = len(gene)
            binarize = []
            labels.append("Gene " + str(row+1))
            thr = onestep(gene)

            for j in range(sizeGene):
                if(gene[j] <= thr):
                    binarize.append(0) 
                else:
                    binarize.append(1)     

            binarize_vect.append(binarize) 
            
        selected = df.iloc[selected_rows]
        genes = selected.values
        
        data = go.Figure(data=go.Heatmap(
                    z=genes,
                    y = labels,
                    text=binarize_vect,
                    texttemplate="%{text}",
                    textfont={"size":20}))
        data.update_layout(
            title={
        'text': "Heatmap of Binarized Genes using Onestep",
            }
        )
        return dcc.Graph(figure=data)
    
    elif(selected_method == "Shmulevich"):
        binarize_vect = []
        labels = []

        for row in selected_rows:
            selected = df.iloc[row]
            gene = selected.values
            sizeGene = len(gene)
            binarize = []
            labels.append("Gene " + str(row+1))
            thr = shmulevich(gene)

            for j in range(sizeGene):
                if(gene[j] <= thr):
                    binarize.append(0) 
                else:
                    binarize.append(1)     

            binarize_vect.append(binarize) 
            selected = df.iloc[selected_rows]
            genes = selected.values

            data = go.Figure(data=go.Heatmap(
                        z=genes,
                        y = labels,
                        text=binarize_vect,
                        texttemplate="%{text}",
                        textfont={"size":20}))
            data.update_layout(
                title={
            'text': "Heatmap of Binarized Genes using Shmulevich",
                }
            )
            return dcc.Graph(figure=data)

@app.callback(
    Output('select-gene-binarize', 'children'),
    Input('dropdown-method', 'value'),
    Input('datatable-interactivity', 'selected_rows'), prevent_initial_call=True)
def select_gene_binarize(selected_method, selected_rows):
    if selected_method is None:
        return None

    if not selected_rows:
        return None

    return [html.H5('Select gene to visualize:'), dcc.Dropdown(
        options=[{'label': 'Gene ' + str(row+1), 'value': row} for row in selected_rows],
        placeholder="Select rows",
        id="dropdown-selected-rows")]

@app.callback(
    Output('graph-gene-binarize', 'children'),
    Input('dropdown-method', 'value'),
    Input('dropdown-selected-rows', 'value'), prevent_initial_call=True)
def graph_gene_algorithm(selected_method, selected_gene):
    if not selected_gene:
        return None
    
    selected = df.iloc[selected_gene]
    gene = selected.values
    sizeGene = len(gene)
    
    if(selected_method == 'BASC A'):
        thr , _ = BASC_A(gene)
        data = go.Figure(go.Scatter(x=np.arange(1,sizeGene+1), y=gene, name="Gene "+ str(selected_gene+1)))
        data.add_hline(y=thr, line_width=3, line_dash="dash", line_color="green")
        
        data.update_layout(
            title={
        'text': "BASC A Threshold of Gene " + str(selected_gene+1),
            }
        )
        return dcc.Graph(figure=data)
    
    elif(selected_method == 'Onestep'):
        thr = onestep(gene)
        data = go.Figure(go.Scatter(x=np.arange(1,sizeGene+1), y=gene, name="Gene "+ str(selected_gene+1)))
        data.add_hline(y=thr, line_width=3, line_dash="dash", line_color="green")
        
        data.update_layout(
            title={
        'text': "Onestep Threshold of Gene " + str(selected_gene+1),
            }
        )
        return dcc.Graph(figure=data)
    
    elif(selected_method == 'Shmulevich'):
        thr = shmulevich(gene)
        data = go.Figure(go.Scatter(x=np.arange(1,sizeGene+1), y=gene, name="Gene "+ str(selected_gene+1)))
        data.add_hline(y=thr, line_width=3, line_dash="dash", line_color="green")
        
        data.update_layout(
            title={
        'text': "Shmulevich Threshold of Gene " + str(selected_gene+1),
            }
        )
        return dcc.Graph(figure=data)
        
    elif(selected_method == 'K-Means'):
        thr = K_Means(gene)
        data = go.Figure(go.Scatter(x=np.arange(1,sizeGene+1), y=gene, name="Gene "+ str(selected_gene+1)))
        data.add_hline(y=thr, line_width=3, line_dash="dash", line_color="green")
        
        data.update_layout(
            title={
        'text': "K-Means Threshold of Gene " + str(selected_gene+1),
            }
        )
        
        data2 = go.Figure(go.Scatter(x=gene, y=np.ones(sizeGene, dtype=int),mode = 'markers'))
        data2.add_vline(x=thr, line_width=3, line_dash="dash", line_color="green")
        
        data2.update_layout(
            title={
        'text': "K-Means Clusters of Gene " + str(selected_gene+1),
            }
        )
        
        return [dcc.Graph(figure=data), dcc.Graph(figure=data2)]
    
    elif(selected_method == "All"):
        thrBasc = BASC_A(gene)
        thrKmeans = K_Means(gene)
        thrShmu = shmulevich(gene)
        thrOnestep = onestep(gene)
        
        data = go.Figure(go.Scatter(x=np.arange(1,sizeGene+1), y=gene, name="Gene "+ str(selected_gene+1)))
        data.add_hline(y=thrBasc, line_width=3, line_dash="dash", line_color="green")
        data.add_annotation(xref="paper", y=thrBasc, text="BASC A", showarrow=False)
        
        data.add_hline(y=thrKmeans, line_width=3, line_dash="dash", line_color="red")
        data.add_annotation(xref="paper", y=thrKmeans, text="KMeans", showarrow=False)
        
        data.add_hline(y=thrOnestep, line_width=3, line_dash="dash", line_color="orange")
        data.add_annotation(xref="paper", y=thrOnestep, text="OneStep", showarrow=False)
        
        data.add_hline(y=thrShmu, line_width=3, line_dash="dash", line_color="purple")
        data.add_annotation(xref="paper", y=thrShmu, text="Shmulevich", showarrow=False)
        
        data.update_layout(
            title={
        'text': "Threshold of Gene " + str(selected_gene+1) + " using all methods",
            }
        )
            
        return dcc.Graph(figure=data)
    
@app.callback(
    Output('select-basc-discontinuity', 'children'),
    Input('dropdown-method', 'value'),
    Input('dropdown-selected-rows', 'value'), prevent_initial_call=True)
def select_basc_discontinuity(selected_method, selected_gene):
    if selected_method is None:
        return None

    if not selected_gene:
        return None
    
    if(selected_method == 'BASC A'):
    
        selected = df.iloc[selected_gene]
        gene = selected.values
        sizeGene = len(gene)
        
        options = [{'label':'All','value':0}]
        for i in range(sizeGene-2):
            options.append({'label':'Discontinuity ' + str(i+1), 'value': i+1})
            
        return [html.H5('Select discontinuity to visualize:'), dcc.Dropdown(
            options=options,
            placeholder="Select discontinuity",
            id="dropdown-selected-discontinuity", value=0)]
    
@app.callback(
    Output('graph-gene-discontinuity', 'children'),
    Input('dropdown-selected-rows', 'value'),
    Input('dropdown-selected-discontinuity', 'value'), prevent_initial_call=True)
def graph_discontinuity(selected_gene, selected_discontinuity):
    if selected_discontinuity is None:
        return None

    if not selected_gene:
        return None
    
    selected = df.iloc[selected_gene]
    gene = np.sort(selected.values)
    sizeGene = len(gene)
    
    if(selected_discontinuity > 0):
            _ , P = BASC_A(gene) 
            
            sizeP = len(P)
            y = np.sort(gene)

            x_dis = []
            y_dis = []

            for i in range(sizeP):
                if(P[i][selected_discontinuity-1]>0):

                    index = P[i][selected_discontinuity-1]

                    if not x_dis:
                        x_dis.append(0)
                    else:
                        x_dis.append(P[i-1][selected_discontinuity-1])
                    x_dis.append(index)
                    x_dis.append(None)

                    y_dis.append(y[index-1])
                    y_dis.append(y[index-1])
                    y_dis.append(None)
                    #print(index)

            x_dis.append(index)
            x_dis.append(len(gene))
            x_dis.append(None)

            y_dis.append(y[index])
            y_dis.append(y[index])
            y_dis.append(None)

            data2 = go.Figure(go.Scatter(x=x_dis, y=y_dis))
            
            data2.update_layout(
            title={
                'text': "BASC A Discontinuity " + str(selected_discontinuity) + " of Gene " + str(selected_gene+1),
                    }
                )
            
            return dcc.Graph(figure=data2)
                
    else:
        x = np.arange(1,sizeGene+1)
        y = gene
        x_dis = []
        y_dis = []
        for i in range(len(x)):
                x_dis.append(i)
                x_dis.append(i+1)
                x_dis.append(None)
                
                y_dis.append(y[i])
                y_dis.append(y[i])
                y_dis.append(None)
            
        data2 = go.Figure(go.Scatter(x=x_dis, y=y_dis))
        
        data2.update_layout(
            title={
                'text': "BASC A Discontinuities of Gene " + str(selected_gene+1),
                    })
            
        return dcc.Graph(figure=data2)
    
@app.callback(
    Output('interpolation-dropdown', 'children'),
    Input('dropdown-selected-rows', 'value'),
    Input('dropdown-method', 'value'), prevent_initial_call=True)
def interpolation_dropdown(selected_row, selected_method):
    if not selected_row:
        return None
    return  [html.H5('Select sample size for interpolation:'), 
            dcc.Dropdown(
                [100, 200, 300, 400, 500],
                placeholder="Select sample size",
                id="dropdown-interpolation",
                searchable=False)]

@app.callback(
    Output('tolerance-dropdown', 'children'),
    Input('dropdown-selected-rows', 'value'),
    Input('dropdown-method', 'value'), prevent_initial_call=True)
def interpolation_dropdown(selected_row, selected_method):
    if not selected_row:
        return None
    return  [html.H5('Select tolerance for interpolation:'), 
            dcc.Dropdown(
                [0.1, 0.01, 0.001, 0.0001, 0.00001],
                placeholder="Select tolerance",
                id="dropdown-tolerance",
                searchable=False)]


@app.callback(
    Output('interpolation-graph', 'children'),
    Input('dropdown-selected-rows', 'value'),
    Input('dropdown-method', 'value'),
    Input('dropdown-interpolation', 'value'),
    Input('dropdown-tolerance', 'value'), prevent_initial_call=True)
def interpolation_dropdown(selected_row, selected_method, sample_size, tolerance):
    if not selected_row:
        return None
    if not sample_size:
        return None
    if not tolerance:
        return None
    
    if(selected_method != 'All'):
        
        thr = 0
        selected = df.iloc[selected_row]
        gene = selected.values
        sizeGene = len(gene)
        
        thrMin, thrMax = interpolation(gene, selected_method, tolerance, sample_size)
        
        #return thrMin, thrMax
        
        if(selected_method == 'K-Means'):
            thr = K_Means(gene)
            
        elif(selected_method == 'BASC A'):
            thr, _ = BASC_A(gene)
            
        elif(selected_method == 'Onestep'):
            thr = onestep(gene)
            
        else:
            thr = shmulevich(gene)

        data = go.Figure(go.Scatter(x=np.arange(1,sizeGene+1), y=gene, name="Gene "+ str(selected_row+1) + 
                                    " interpolation of sample size " + str(sample_size) 
                                    + " and tolerance " + str(tolerance)))
        data.add_hline(y=thr, line_width=3, line_dash="dash", line_color="green")
        data.add_hline(y=thrMin, line_width=3, line_dash="dash", line_color="red")
        data.add_hline(y=thrMax, line_width=3, line_dash="dash", line_color="red")
        
        data.update_layout(
            title={
        'text': "Gene "+ str(selected_row+1) + 
                                    " interpolation of sample size " + str(sample_size) 
                                    + " and tolerance " + str(tolerance),
            }
        )
        return dcc.Graph(figure=data)
    
    else:
        return None


if __name__ == '__main__':
    app.run_server(debug=True)

C:\Python311\Lib\site-packages\sklearn\cluster\_kmeans.py:1416: FutureWarning:

The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning

C:\Python311\Lib\site-packages\numpy\core\fromnumeric.py:3432: RuntimeWarning:

Mean of empty slice.

C:\Python311\Lib\site-packages\numpy\core\_methods.py:190: RuntimeWarning:

invalid value encountered in double_scalars

C:\Python311\Lib\site-packages\sklearn\cluster\_kmeans.py:1416: FutureWarning:

The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning

C:\Python311\Lib\site-packages\numpy\core\fromnumeric.py:3432: RuntimeWarning:

Mean of empty slice.

C:\Python311\Lib\site-packages\numpy\core\_methods.py:190: RuntimeWarning:

invalid value encountered in double_scalars

C:\Python311\Lib\site-packages\sklearn\cluster\_kmeans.py:1416: FutureWarning:

The default value of `n_init` will change from 10 to '

11724.266666666668
5888611.5333333295
55779.916666666664
58290117.188333265
25940.816666666666
4671934.148333332
5.31982
1.695963721
6714.100000000001
999192.0296000004
1541.87
865409.9948
9967.195
14538707.032350004
38969.4
17451122.599999994
12728.85
3889002.915000001
38.16095
792.5356314950002


C:\Python311\Lib\site-packages\numpy\core\fromnumeric.py:3432: RuntimeWarning:

Mean of empty slice.

C:\Python311\Lib\site-packages\numpy\core\_methods.py:190: RuntimeWarning:

invalid value encountered in double_scalars

C:\Python311\Lib\site-packages\sklearn\cluster\_kmeans.py:1416: FutureWarning:

The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning

C:\Python311\Lib\site-packages\numpy\core\fromnumeric.py:3432: RuntimeWarning:

Mean of empty slice.

C:\Python311\Lib\site-packages\numpy\core\_methods.py:190: RuntimeWarning:

invalid value encountered in double_scalars

C:\Python311\Lib\site-packages\sklearn\cluster\_kmeans.py:1416: FutureWarning:

The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning

C:\Python311\Lib\site-packages\numpy\core\fromnumeric.py:3432: RuntimeWarning:

Mean of empty slice.

C:\Python311\Lib\site-packages\n

13722.966666666667
2807582.2333333353
14392.65
2489223.1350000016
18516.25
9180855.094999993
36219.51666666667
28811848.96833332
5.881401666666666
0.013843585083333335
40999.73333333333
28141609.77333334
18466.316666666666
10608495.028333327
2571.3483333333334
401629.6586833333
9821.186666666666
3037105.8411333296
21820.083333333336
9096213.828333335
23290.933333333334
4290965.293333334
4.756891666666667
1.9604182050833328
14994.116666666667
840294.6283333328
18110.45
4884226.895000003
70559.46666666666
10982201.493333345


C:\Python311\Lib\site-packages\numpy\core\fromnumeric.py:3432: RuntimeWarning:

Mean of empty slice.

C:\Python311\Lib\site-packages\numpy\core\_methods.py:190: RuntimeWarning:

invalid value encountered in double_scalars

C:\Python311\Lib\site-packages\sklearn\cluster\_kmeans.py:1416: FutureWarning:

The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning

C:\Python311\Lib\site-packages\numpy\core\fromnumeric.py:3432: RuntimeWarning:

Mean of empty slice.

C:\Python311\Lib\site-packages\numpy\core\_methods.py:190: RuntimeWarning:

invalid value encountered in double_scalars

C:\Python311\Lib\site-packages\sklearn\cluster\_kmeans.py:1416: FutureWarning:

The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning

C:\Python311\Lib\site-packages\numpy\core\fromnumeric.py:3432: RuntimeWarning:

Mean of empty slice.

C:\Python311\Lib\site-packages\n

...

6.096203333333334
18.988101188933335
10.190995000000001
28.26034945915001
14226.75
22816294.534999996
24520.11666666667
24452121.02833333
7.088711666666666
45.37845284188333
3.3273899999999994
0.01719701240000003
8.013119999999999
13.353270708999997
5.07108
4.426092829199997
10891.671666666665
33102043.080683336
46.91791666666666
1405.0965634083332


C:\Python311\Lib\site-packages\sklearn\cluster\_kmeans.py:1416: FutureWarning:

The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning

C:\Python311\Lib\site-packages\numpy\core\fromnumeric.py:3432: RuntimeWarning:

Mean of empty slice.

C:\Python311\Lib\site-packages\numpy\core\_methods.py:190: RuntimeWarning:

invalid value encountered in double_scalars

C:\Python311\Lib\site-packages\sklearn\cluster\_kmeans.py:1416: FutureWarning:

The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning

C:\Python311\Lib\site-packages\numpy\core\fromnumeric.py:3432: RuntimeWarning:

Mean of empty slice.

C:\Python311\Lib\site-packages\numpy\core\_methods.py:190: RuntimeWarning:

invalid value encountered in double_scalars

C:\Python311\Lib\site-packages\sklearn\cluster\_kmeans.py:1416: FutureWarning:

The default value of `n_init` will change from 10 to '

9708.269999999999
7887301.817200001
5.306541666666667
4.071505497083333
5.513114999999999
0.51593825575
29549.46666666667
8189661.333333327
16375.516666666668
6312827.22833334
22161.050000000003
5812908.774999991
47661.1
24822667.180000015


C:\Python311\Lib\site-packages\numpy\core\fromnumeric.py:3432: RuntimeWarning:

Mean of empty slice.

C:\Python311\Lib\site-packages\numpy\core\_methods.py:190: RuntimeWarning:

invalid value encountered in double_scalars

C:\Python311\Lib\site-packages\sklearn\cluster\_kmeans.py:1416: FutureWarning:

The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning

C:\Python311\Lib\site-packages\numpy\core\fromnumeric.py:3432: RuntimeWarning:

Mean of empty slice.

C:\Python311\Lib\site-packages\numpy\core\_methods.py:190: RuntimeWarning:

invalid value encountered in double_scalars

C:\Python311\Lib\site-packages\sklearn\cluster\_kmeans.py:1416: FutureWarning:

The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning

C:\Python311\Lib\site-packages\numpy\core\fromnumeric.py:3432: RuntimeWarning:

Mean of empty slice.

C:\Python311\Lib\site-packages\n

16230.35
5589266.295000007
15746.666666666666
4740021.733333327
18235.0
12078290.439999992
45898.53333333333
3933580.793333343
60513.399999999994
23380241.320000008
25.513933333333338
42.08864297333333
20132.06666666667
9051204.793333339


# Code example

In [13]:
gene = [0.22,0.29,0.1,0.13,0.8,0.9,0.22,0.85,0.81,0.5]


In [7]:
onestep(gene)

0.5111111111111112
0.5387500000000001
0.6014285714285715
0.68
0.656
0.595
0.7200000000000001
0.655
0.5


0.4325

In [21]:
interpolationConverge(gene,"", 0.1)

(0.0548081630175201, 1.002243550085137, 0.9087524149376105, 10000)